In [1]:
#!pip install faiss-gpu
#!pip install openpyxl
#!pip install --upgrade ollama
#!pip install openpyxl

In [2]:
from ollama import chat
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, ValidationError
import pandas as pd
from pandas import json_normalize
import json
import os
import glob
import re
from typing import Optional, Literal
import collections

In [3]:
loader = PDFPlumberLoader("docs/urgencias/Hist6.pdf")
docs = loader.load()

# Check the number of pages
print("Number of pages in the PDF:",len(docs))

Number of pages in the PDF: 2


In [4]:
class InformacionPaciente(BaseModel):
    id_episodio: str = Field(description="Clave identificadora del episodio o historia clínica (NºHª)")
    fecha_nacimiento: str = Field(description="Fecha de nacimiento del paciente")
    edad: int = Field(description="Edad del paciente")
    sexo: str = Field(description="Sexo del paciente")
    fecha_ingreso_hospital: str = Field(description="Fecha de ingreso en el hospital")
    fecha_ingreso_uci: str = Field(description="Fecha de ingreso en la UCI")
    hora_ingreso_uci: str = Field(description="Hora de ingreso en la UCI")
    estancia_preuci: int = Field(description="Número de días en los que ha estado hospitalizado antes de entrar en la UCI")
    reingreso: bool = Field(description="¿Ha sido ingresado el paciente con anterioridad?")
    numero_ingresos_previos: int = Field(description="Número de veces que el paciente ha sido ingresado con anterioridad")
    tipo_ingreso_uci: str = Field(description="¿Ha sido el ingreso en UCI del paciente de tipo programado o de tipo urgente?")
    tipo_paciente: str = Field(description="¿Se trata de un paciente de tipo médico o de tipo quirúrgico?")
    procedencia: str = Field(description="Indíquese si el origen del paciente ha sido el quirófano, una hospitalización convencional, la sala de urgencias u otro área hospitalaria")
    fecha_alta_uci: str = Field(description="Fecha de alta de la UCI")
    hora_alta_uci: str = Field(description="Hora de alta de la UCI")
    estancia_uci: int = Field(description="Número de días que el paciente ha estado en la UCI")
    estancia_postuci: str = Field(description="Número de días que el paciente ha estado hospitalizado después de salir de la UCI")
    destino_alta_uci: str = Field(description="Destino del paciente tras haber sido dado de alta en la UCI: exitus letalis, envío a domicilio, traslado a otro centro hospitalario u hospitalización convencional")
    estado_paciente: str = Field(description="Estado del paciente tras haber sido dado de alta del hospital: vivo o fallecido")
    motivo_ingreso: str = Field(description="Motivo de ingreso en la UCI")
    diagnostico: str = Field(description="Diagnóstico principal de acuerdo con la CIE (indíquese la enfermedad con su código, como por ejemplo aquí: D42 - Neoplasia de comportamiento incierto de meninges)")


In [5]:
InformacionPaciente.update_forward_refs()

/tmp/ipykernel_3449441/2475997593.py:1: PydanticDeprecatedSince20: The `update_forward_refs` method is deprecated; use `model_rebuild` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  InformacionPaciente.update_forward_refs()


In [6]:
llm_to_use ="mistral"

In [7]:
text_splitter = SemanticChunker(OllamaEmbeddings(model=llm_to_use, base_url="http://127.0.0.1:11434"))
documents = text_splitter.split_documents(docs)

In [8]:
# Paso 1: Configuración del parser para la salida estructurada con Pydantic
#parser = PydanticOutputParser(pydantic_object=InformacionPaciente)

# Paso 2: Configuración del embedder y el vector retriever
embedder = OllamaEmbeddings(model=llm_to_use, base_url="http://127.0.0.1:11434")
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Paso 3: Configuración del modelo LLM
llm = Ollama(model=llm_to_use, temperature=0)


/tmp/ipykernel_3449441/98834687.py:10: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=llm_to_use, temperature=0)


In [ ]:
consulta = "Obtener la información del paciente"
# Recuperar documentos relevantes
documentos_relevantes = retriever.get_relevant_documents(consulta)
if not documentos_relevantes:
    print("No se encontraron documentos relevantes para la consulta.")

print()

# Formatear el contexto con los documentos recuperados
historia_clinica = "\n\n".join([doc.page_content for doc in documentos_relevantes if hasattr(doc, 'page_content') and doc.page_content])
if not historia_clinica:
    print("Historia clínica vacía después de procesar los documentos.")

print()

# Generar el esquema JSON
#formato_resultado = InformacionPaciente.model_json_schema()
#print("Esquema JSON generado:", formato_resultado)

print()

# Crear los mensajes con el contexto
def extraer_campo(historia_clinica, campo, descripcion):
    response = chat(
        messages=[
            {
                'role': 'user',
                'content': (
                    f"Extrae el siguiente dato de la historia clínica: {descripcion}\n\n"
                    f"Historia clínica:\n{contexto}"
                )
            }
        ],
        model="llama3.2"
    )

    print(campo)
    print(descripcion)
    print(response.message.content)
    return response.message.content

/tmp/ipykernel_3449441/3822547142.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  documentos_relevantes = retriever.get_relevant_documents(consulta)


In [ ]:
# Extraer cada campo
datos_paciente = {campo: extraer_campo(historia_clinica, campo, descripcion.description) for campo, descripcion in InformacionPaciente.model_fields.items()}

df = pd.DataFrame([datos_paciente])

# Guardar en un archivo Excel en una hoja con nombre específico
archivo_excel = "docs/urgencias/ResultadosLlama3_2.xlsx"
nombre_hoja = "SinPydantic"

try:
    with pd.ExcelWriter(archivo_excel, mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
        existing_df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)
        df_final = pd.concat([existing_df, df], ignore_index=True)
        df_final.to_excel(writer, index=False, sheet_name=nombre_hoja)
except FileNotFoundError:
    with pd.ExcelWriter(archivo_excel, mode="w", engine="openpyxl") as writer:
        df.to_excel(writer, index=False, sheet_name=nombre_hoja)

print("Datos del paciente extraídos y guardados en la hoja específica del Excel correctamente.")


id_episodio
Clave identificadora del episodio o historia clínica (NºHª)
 El número de historia clínica (NºHª) es el 0325476868.
fecha_nacimiento
Fecha de nacimiento del paciente
 El dato que se necesita es la fecha de nacimiento del paciente, pero en esta historia clínica no se proporciona. En los antecedentes personales solo se mencionan las alergias y otras enfermedades, pero no su edad. Para obtener la información más precisa, es necesario consultar otros documentos médicos del paciente que contengan la fecha de nacimiento o edad.
edad
Edad del paciente
 El paciente es Pepito Pérez Cuadrado (Paciente). No se encuentra la edad específica en la historia clínica proporcionada, por lo tanto no puedo extraer ese dato específico. Sin embargo, tenemos su fecha de ingreso en UCI: 20/04/2021 y su fecha de alta de UCI: 21/04/21, a partir de los cuales se puede inferir que tiene un año completo si el periodo entre ambas fechas es inferior a un año. Por otra parte, la edad se menciona en el inf

In [11]:
df

,id_episodio,fecha_nacimiento,edad,sexo,fecha_ingreso_hospital,fecha_ingreso_uci,hora_ingreso_uci,estancia_preuci,reingreso,numero_ingresos_previos,...,tipo_paciente,procedencia,fecha_alta_uci,hora_alta_uci,estancia_uci,estancia_postuci,destino_alta_uci,estado_paciente,motivo_ingreso,diagnostico
0,El número de historia clínica (NºHª) es el 03...,El dato que se necesita es la fecha de nacimi...,El paciente es Pepito Pérez Cuadrado (Pacient...,1. Brilique 90 mg VO/12h (Medicación)\n2. Enox...,El dato que necesitas extraer de la historia ...,El dato extraído de la historia clínica es la...,La hora de ingreso en la UCI no se encuentra ...,El número de días en los que ha estado hospit...,En la historia clínica no se menciona si el p...,"Desafortunadamente, no se proporciona informa...",...,El paciente no se especifica como médico o qu...,El paciente ha sido procedente de la sala de ...,La fecha de alta de la UCI es 21/04/21 (Día 2...,Fecha de alta de la UCI: 21/04/21,El paciente se ingresó en la UCI el día 20/04...,El paciente ha estado hospitalizado después d...,Destino del paciente tras haber sido dado de ...,Estado del paciente tras haber sido dado de a...,"El motivo de ingreso en la UCI es ""paciente d...",El diagnóstico principal de acuerdo con la CI...


file_path = "docs/urgencias/ResultadosPhi3.xlsx"
sheet_name = "SinPydantic"  # Nombre de la nueva hoja

try:
    # Verificar si el archivo existe antes de cargarlo
    if os.path.exists(file_path):
        df_existente = pd.read_excel(file_path, sheet_name=None)  # Cargar todas las hojas como diccionario de DataFrames
    else:
        df_existente = {}  # Si el archivo no existe, inicializar como diccionario vacío

    # Convertir JSON a DataFrame
    data = json.loads(resultado)  # Asegurar que 'resultado' es un JSON válido
    flattened_data = pd.json_normalize(data)  # Normalizar JSON
    df_nuevo = pd.DataFrame(flattened_data)

    # Obtener el DataFrame existente en la hoja si ya existe, si no, dejarlo vacío
    df_hoja_existente = df_existente.get(sheet_name, pd.DataFrame())

    # Concatenar los datos nuevos con los existentes
    df_actualizado = pd.concat([df_hoja_existente, df_nuevo], ignore_index=True)

    # Guardar en una nueva hoja sin sobrescribir las existentes
    with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        df_actualizado.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Datos actualizados exitosamente en la hoja '{sheet_name}'.")

except Exception as e:
    print(f"Error: {e}")